# Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

![avatar](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

# Цели проекта

Для формулирования ценностного предложения для нового продукта мы должны исследовать предоставленные данные, а именно:
* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем из анализа брошюры);
* Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Загрузка и обзор данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Подключаемся к базе.

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Загрузим предоставленные датасеты.

In [3]:
q_books = ''' SELECT * FROM books; ''' 
books = pd.io.sql.read_sql(q_books, con = engine)

q_authors = ''' SELECT * FROM authors; ''' 
authors = pd.io.sql.read_sql(q_authors, con = engine)

q_ratings = ''' SELECT * FROM ratings; ''' 
ratings = pd.io.sql.read_sql(q_ratings, con = engine)

q_reviews = ''' SELECT * FROM reviews; ''' 
reviews = pd.io.sql.read_sql(q_reviews, con = engine)

q_publishers = ''' SELECT * FROM publishers; ''' 
publishers = pd.io.sql.read_sql(q_publishers, con = engine)

Напишем функцию `inf` для вывода определенной информации о датасете.

In [4]:
def inf(data: 'DataFrame'):
    print()
    print ('\033[1m' + 'Описание данных в таблице' + '\033[0m')    
    print(data.info())
    print('')
    print ('\033[1m' + 'Процент пропусков в таблице' + '\033[0m')    
    print(data.isna().mean())
    print('')
    print ('\033[1m' + 'Проверка на дубликаты' + '\033[0m')    
    display(data.duplicated().sum())
    print('')
    print('\033[1m' + 'Основные статистические данные' + '\033[0m')
    display(data.describe())    
    print('')
    print ('\033[1m' + 'Визуальный осмотр данных таблицы' + '\033[0m')          
    display(data.head())

### Датасет `books`

In [5]:
inf(books)


Описание данных в таблице
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None

Процент пропусков в таблице
book_id             0.0
author_id           0.0
title               0.0
num_pages           0.0
publication_date    0.0
publisher_id        0.0
dtype: float64

Проверка на дубликаты


0


Основные статистические данные


,book_id,author_id,num_pages,publisher_id
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,320.417000,389.11100,171.270000
std,288.819436,181.620172,229.39014,99.082685
min,1.000000,1.000000,14.00000,1.000000
25%,250.750000,162.750000,249.00000,83.000000
50%,500.500000,316.500000,352.00000,177.500000
75%,750.250000,481.000000,453.00000,258.000000
max,1000.000000,636.000000,2690.00000,340.000000



Визуальный осмотр данных таблицы


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Данные чистые. При необходимости поменяем тип переменной `publication_date` на `datetime` в запросах.

### Датасет `authors`

In [6]:
inf(authors)


Описание данных в таблице
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None

Процент пропусков в таблице
author_id    0.0
author       0.0
dtype: float64

Проверка на дубликаты


0


Основные статистические данные


,author_id
count,636.000000
mean,318.500000
std,183.741666
min,1.000000
25%,159.750000
50%,318.500000
75%,477.250000
max,636.000000



Визуальный осмотр данных таблицы


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Данные чистые.

### Датасет `ratings`

In [7]:
inf(ratings)


Описание данных в таблице
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None

Процент пропусков в таблице
rating_id    0.0
book_id      0.0
username     0.0
rating       0.0
dtype: float64

Проверка на дубликаты


0


Основные статистические данные


,rating_id,book_id,rating
count,6456.000000,6456.000000,6456.000000
mean,3228.500000,510.574195,3.928284
std,1863.831001,284.141636,0.943303
min,1.000000,1.000000,1.000000
25%,1614.750000,291.000000,3.000000
50%,3228.500000,506.000000,4.000000
75%,4842.250000,750.000000,5.000000
max,6456.000000,1000.000000,5.000000



Визуальный осмотр данных таблицы


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Данные чистые.

### Датасет `reviews`

In [8]:
inf(reviews)


Описание данных в таблице
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None

Процент пропусков в таблице
review_id    0.0
book_id      0.0
username     0.0
text         0.0
dtype: float64

Проверка на дубликаты


0


Основные статистические данные


,review_id,book_id
count,2793.000000,2793.000000
mean,1397.000000,504.693161
std,806.413976,288.472931
min,1.000000,1.000000
25%,699.000000,259.000000
50%,1397.000000,505.000000
75%,2095.000000,753.000000
max,2793.000000,1000.000000



Визуальный осмотр данных таблицы


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Датасет `publishers`

In [9]:
inf(publishers)


Описание данных в таблице
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None

Процент пропусков в таблице
publisher_id    0.0
publisher       0.0
dtype: float64

Проверка на дубликаты


0


Основные статистические данные


,publisher_id
count,340.000000
mean,170.500000
std,98.293777
min,1.000000
25%,85.750000
50%,170.500000
75%,255.250000
max,340.000000



Визуальный осмотр данных таблицы


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Данные чистые.

### Вывод

На этом этапе проекта мы получили следующие результаты:
* подключились к базе;
* загрузили предоставленные датасеты;
* произвели обзор данных, в ходе которого:
    * посмотрели на первые строки каждого из датасетов;
    * убедились, что данные чистые;
    * в датасете `books` присутствует переменная `publication_date`, тип которой можно поменять на `datetime` в запросах, при необходимости.

## Запросы

### Посчитать, сколько книг вышло после 1 января 2000 года

In [10]:
request_1 = '''
SELECT 
    COUNT(book_id) AS nbooks
FROM
    books
WHERE
    publication_date > '2000-01-01'
'''

r_1 = pd.io.sql.read_sql(request_1, con = engine)
r_1

,nbooks
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитать количество обзоров и среднюю оценку

In [11]:
request_2 = '''
SELECT
    title,
    rating_avg,
    review_cnt
FROM
    books 
    LEFT JOIN 
            (SELECT
                book_id,
                AVG(rating) AS rating_avg
            FROM
                ratings
            GROUP BY
                book_id) AS s_ratings ON books.book_id = s_ratings.book_id 
    LEFT JOIN 
            (SELECT
                book_id,
                COUNT(review_id) AS review_cnt
            FROM
                reviews
            GROUP BY
                book_id) AS s_reviews ON books.book_id = s_reviews.book_id
'''

r_2 = pd.io.sql.read_sql(request_2, con = engine)
r_2

,title,rating_avg,review_cnt
0,The Body in the Library (Miss Marple #3),4.500000,2.0
1,Galápagos,4.500000,2.0
2,A Tree Grows in Brooklyn,4.250000,5.0
3,Undaunted Courage: The Pioneering First Missio...,4.000000,2.0
4,The Prophet,4.285714,4.0
...,...,...,...
995,Alice in Wonderland,4.230769,4.0
996,A Woman of Substance (Emma Harte Saga #1),5.000000,2.0
997,Christine,3.428571,3.0
998,The Magicians' Guild (Black Magician Trilogy #1),3.500000,2.0


### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем из анализа брошюры)

In [12]:
request_3 = '''
SELECT
    publisher,
    COUNT(book_id) as book_cnt
FROM
    books 
    LEFT JOIN 
            (SELECT
                publisher_id,
                publisher
            FROM
                publishers) AS s_publishers ON books.publisher_id = s_publishers.publisher_id

WHERE
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    book_cnt DESC
LIMIT 1
'''

r_3 = pd.io.sql.read_sql(request_3, con = engine)
r_3

,publisher,book_cnt
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг - **Penguin Books** (42 книги).

### Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками


In [13]:
request_4 = '''
SELECT
    author,
    CASE
        WHEN AVG(rating_avg) IS NULL THEN 0
        ELSE AVG(rating_avg)
    END AS rating_author
FROM
    books 
    LEFT JOIN 
            (SELECT
                author_id,
                author
            FROM
                authors) AS s_author ON books.author_id = s_author.author_id
    INNER JOIN 
            (SELECT
                book_id,
                AVG(rating) as rating_avg,
                COUNT(rating) AS rating_cnt
            FROM
                ratings
            GROUP BY
                book_id
            HAVING
                COUNT(rating) >= 50) AS s_ratings ON books.book_id = s_ratings.book_id
GROUP BY
    author
ORDER BY
    rating_author DESC
LIMIT 1
'''

r_4 = pd.io.sql.read_sql(request_4, con = engine)
r_4

,author,rating_author
0,J.K. Rowling/Mary GrandPré,4.283844


Автором с самой высокой средней оценкой книг является **J.K. Rowling/Mary GrandPré**(4.28).

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [14]:
request_5 = '''
SELECT
    AVG(review_cnt)
FROM
    (SELECT
        reviews.username,
        COUNT(review_id) AS review_cnt
     FROM
        reviews
     INNER JOIN
         (SELECT
             username
          FROM
              ratings
          GROUP BY
              username
          HAVING
              COUNT(rating_id) > 50) AS s_ratings ON reviews.username = s_ratings.username
          GROUP BY
              reviews.username) AS t
'''

r_5 = pd.io.sql.read_sql(request_5, con = engine)
r_5

,avg
0,24.333333


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок, равно 24.33.

## Общий вывод

После подключения к базе и загрузки датасетов было совершено 5 запросов, из которых были получены следующие результаты:
* После 1 января 2000 года вышло 819 книг;
* Для каждой книги было посчитано количество обзоров и средняя оценка;
* Издательство, которое выпустило наибольшее число книг - **Penguin Books** (42 книги);
* Автором с самой высокой средней оценкой книг является **J.K. Rowling/Mary GrandPré**(4.28);
* Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок, равно 24.33.